In [1]:
import dask.array as da
import numpy as np
import pyproj as pp
import xarray as xr

In [2]:
ds = xr.open_zarr('../inputdata/S3-OLCI-L2A.zarr.zip')
ds

C:\Users\norma\AppData\Local\Temp\ipykernel_8596\1658381369.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_zarr('../inputdata/S3-OLCI-L2A.zarr.zip')


<xarray.Dataset>
Dimensions:        (y: 1890, x: 1189)
Coordinates:
    lat            (y, x) float64 dask.array<chunksize=(512, 512), meta=np.ndarray>
    lon            (y, x) float64 dask.array<chunksize=(512, 512), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    quality_flags  (y, x) uint32 dask.array<chunksize=(512, 512), meta=np.ndarray>
    rtoa_3         (y, x) float32 dask.array<chunksize=(512, 512), meta=np.ndarray>
    rtoa_6         (y, x) float32 dask.array<chunksize=(512, 512), meta=np.ndarray>
    rtoa_8         (y, x) float32 dask.array<chunksize=(512, 512), meta=np.ndarray>
Attributes:
    Conventions:   CF-1.4
    product_type:  C2RCC_OLCI
    start_date:    04-JUL-2018 09:21:55.677316
    stop_date:     04-JUL-2018 09:23:18.811790

In [3]:
crs_4326 = pp.crs.CRS(4326)
crs_4326

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
crs_4326.is_geographic

True

In [5]:
crs_4326.is_geocentric

False

In [6]:
crs_32633 = pp.crs.CRS(32633)
crs_32633

<Derived Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
t = pp.transformer.Transformer.from_crs(crs_4326, crs_32633)
t

<Conversion Transformer: pipeline>
Description: UTM zone 33N
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore.
- bounds: (12.0, 0.0, 18.0, 84.0)

In [8]:
lon = ds.lon.values
lat = ds.lat.values

In [9]:
%%time
x, y = t.transform(lat, lon)

CPU times: total: 281 ms
Wall time: 514 ms


In [10]:
x, y

(array([[464353.17350437, 464753.99068697, 464753.99068697, ...,
         778366.98327271, 778663.97989779, 778960.82095417],
        [464300.61922255, 464697.86336861, 464697.86336861, ...,
         778311.52553234, 778608.48730308, 778905.40483205],
        [464244.50782782, 464643.5297553 , 464643.5297553 , ...,
         778256.11946402, 778553.04638088, 778849.92904976],
        ...,
        [355582.31759961, 356009.5675282 , 356009.5675282 , ...,
         668917.33742756, 669212.77356855, 669508.22326567],
        [355522.80213548, 355950.07579013, 355950.07579013, ...,
         668857.14267177, 669152.60061632, 669448.00001141],
        [355463.34306198, 355890.64413473, 355890.64413473, ...,
         668796.87484134, 669092.350345  , 669387.77154332]]),
 array([[6722665.0879521 , 6722587.51099095, 6722587.51099095, ...,
         6662383.08675777, 6662325.42149084, 6662267.76727586],
        [6722380.12777498, 6722303.24876914, 6722303.24876914, ...,
         6662095.72015417, 66

In [11]:
coords_4326 = da.stack([ds.lat.data, ds.lon.data]).rechunk((2, None, None))
coords_4326

dask.array<rechunk-merge, shape=(2, 1890, 1189), dtype=float64, chunksize=(2, 512, 512), chunktype=numpy.ndarray>

In [12]:
def transform(block, t=None):
    x1, y1 = block
    x2, y2 = t.transform(x1, y1)
    return np.stack([x2, y2])

In [13]:
coords_32633 = da.apply_gufunc(transform, '()->()', coords_4326, output_dtypes=np.float64, t=t)
coords_32633

dask.array<transpose, shape=(2, 1890, 1189), dtype=float64, chunksize=(2, 512, 512), chunktype=numpy.ndarray>

In [14]:
%%time
x, y = coords_32633.compute()

CPU times: total: 312 ms
Wall time: 247 ms


In [15]:
x, y

(array([[464353.17350437, 464753.99068697, 464753.99068697, ...,
         778366.98327271, 778663.97989779, 778960.82095417],
        [464300.61922255, 464697.86336861, 464697.86336861, ...,
         778311.52553234, 778608.48730308, 778905.40483205],
        [464244.50782782, 464643.5297553 , 464643.5297553 , ...,
         778256.11946402, 778553.04638088, 778849.92904976],
        ...,
        [355582.31759961, 356009.5675282 , 356009.5675282 , ...,
         668917.33742756, 669212.77356855, 669508.22326567],
        [355522.80213548, 355950.07579013, 355950.07579013, ...,
         668857.14267177, 669152.60061632, 669448.00001141],
        [355463.34306198, 355890.64413473, 355890.64413473, ...,
         668796.87484134, 669092.350345  , 669387.77154332]]),
 array([[6722665.0879521 , 6722587.51099095, 6722587.51099095, ...,
         6662383.08675777, 6662325.42149084, 6662267.76727586],
        [6722380.12777498, 6722303.24876914, 6722303.24876914, ...,
         6662095.72015417, 66

In [16]:
def transform2(x1, y1, t=None):
    x2, y2 = t.transform(x1, y1)
    return x2, y2

In [17]:
x, y = da.apply_gufunc(transform2, '(),()->(),()', ds.lat.data, ds.lon.data, output_dtypes=(np.float64,np.float64), t=t)
x, y

(dask.array<transpose, shape=(1890, 1189), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>,
 dask.array<transpose, shape=(1890, 1189), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>)

In [18]:
%%time
xx = x.compute()
yy = y.compute()

CPU times: total: 672 ms
Wall time: 440 ms


In [19]:
xx, yy

(array([[464353.17350437, 464753.99068697, 464753.99068697, ...,
         778366.98327271, 778663.97989779, 778960.82095417],
        [464300.61922255, 464697.86336861, 464697.86336861, ...,
         778311.52553234, 778608.48730308, 778905.40483205],
        [464244.50782782, 464643.5297553 , 464643.5297553 , ...,
         778256.11946402, 778553.04638088, 778849.92904976],
        ...,
        [355582.31759961, 356009.5675282 , 356009.5675282 , ...,
         668917.33742756, 669212.77356855, 669508.22326567],
        [355522.80213548, 355950.07579013, 355950.07579013, ...,
         668857.14267177, 669152.60061632, 669448.00001141],
        [355463.34306198, 355890.64413473, 355890.64413473, ...,
         668796.87484134, 669092.350345  , 669387.77154332]]),
 array([[6722665.0879521 , 6722587.51099095, 6722587.51099095, ...,
         6662383.08675777, 6662325.42149084, 6662267.76727586],
        [6722380.12777498, 6722303.24876914, 6722303.24876914, ...,
         6662095.72015417, 66

In [20]:
pp.crs.CRS.from_string("urn:ogc:def:crs:OGC:1.3:CRS84")

<Geographic 2D CRS: OGC:CRS84>
Name: WGS 84 (CRS84)
Axis Info [ellipsoidal]:
- Lon[east]: Geodetic longitude (degree)
- Lat[north]: Geodetic latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
pp.crs.CRS.from_user_input("epsg:4326")

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich